In [1]:
import open3d as o3d
from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# Loading Lidar readings files (.pcd) files

In [3]:
print("Load a pcd point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud("data/sensors/data_1/0000000000.pcd")
print(pcd)
print(np.asarray(pcd.points))

Load a pcd point cloud, print it, and render it
PointCloud with 119978 points.
[[52.30099869  7.30000019  1.995     ]
 [50.5719986   7.21999979  1.93700004]
 [48.7879982   7.27699995  1.87699997]
 ...
 [ 3.59200001 -1.33800006 -1.671     ]
 [ 3.75600004 -1.38699996 -1.75100005]
 [ 0.          0.          0.        ]]


In [4]:
draw(pcd)

WebVisualizer(window_uid='window_0')

## FIRST PART

### Filtering the initial point cloud. Using voxel grid filtering, we reduce the number of points in the point cloud. 

In [5]:
VOXEL_SIZE = 0.2
print(f"Downsample the point cloud with a voxel of {VOXEL_SIZE}")
downpcd = pcd.voxel_down_sample(voxel_size=VOXEL_SIZE)
draw(downpcd)

Downsample the point cloud with a voxel of 0.2


WebVisualizer(window_uid='window_1')

### Cropping points outside the region of interest.

In [6]:
print("Cropping the image to only left ROI")
bbox = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=(-15, -5, -2), 
    max_bound=(30, 7, 1)
    )
croppcd = downpcd.crop(bbox)
draw(croppcd)

Cropping the image to only left ROI


WebVisualizer(window_uid='window_2')

### Removing points that belong to the "car's" roof.

In [7]:
print("Extracting only the roff points")
roof_bbox = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=(-1.5, -1.7, -1.0), 
    max_bound=(2.6, 1.7, -0.4)
    )
roofpcd = croppcd.crop(roof_bbox)
draw(roofpcd)

Extracting only the roff points


WebVisualizer(window_uid='window_3')

In [8]:
croppcd_points = np.asarray(croppcd.points)
roofpcd_points = np.asarray(roofpcd.points)

indices = []
for roof_element in roofpcd_points:
    array_comparison = np.equal(croppcd_points, roof_element)
    for index, array in enumerate(array_comparison):
        if np.sum(np.logical_and(array, [True, True, True])) == 3:
            indices.append(index)
        
print("Extracting the points that doesn't belong to the roof")
regionpcd = croppcd.select_by_index(indices, invert=True)
draw(regionpcd)

Extracting the points that doesn't belong to the roof


WebVisualizer(window_uid='window_4')

# SECOND STEP

Separating the road from obstacles. We perform Plane segmentation to achieve this goal. Using the RANSAC algorithm we can find the plane with the largest support in the point cloud.

![ransac_raw](./Line_with_outliers.svg)

![ransac_raw](./Fitted_line.svg)

In [9]:
print("Segmententation of geometric primitives from point clouds using RANSAC.")
plane_model, inliers = regionpcd.segment_plane(
    distance_threshold=0.2,
    ransac_n=3,
    num_iterations=100
)

inlier_cloud = regionpcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

draw(inlier_cloud)

Segmententation of geometric primitives from point clouds using RANSAC.


WebVisualizer(window_uid='window_5')

In [10]:
outlier_cloud = regionpcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0.2, 0.5, 0])

draw(outlier_cloud)

WebVisualizer(window_uid='window_6')

## THIRD STEP

Applying clustering to split the input point cloud. We separate each detected obstacle and get its correspondent points. 

In [11]:
print("Clustering using DBSCAN from a point cloud")
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        outlier_cloud.cluster_dbscan(eps=0.7, min_points=10, print_progress=True)
    )

print(labels.shape)
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

Clustering using DBSCAN from a point cloud
(1731,)
point cloud has 9 clusters


In [12]:
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

draw(outlier_cloud)

WebVisualizer(window_uid='window_7')

## FOURTH STEP

Determining and drawing the bounding boxes for the obstacles in the point cloud.

In [13]:
print("Drawing bounding boxes of obstacles")

print(labels.min())

bounding_boxes = []
for cluster_number in list(np.unique(labels))[1:]:
    cluster_indices = np.where(labels == cluster_number)
    cluster_cloud_points = outlier_cloud.select_by_index(
        cluster_indices[0].tolist()
        )
    object_bbox = cluster_cloud_points.get_axis_aligned_bounding_box()
    bounding_boxes.append(object_bbox)

print(len(bounding_boxes))

draw([inlier_cloud, outlier_cloud, *bounding_boxes])

Drawing bounding boxes of obstacles
-1
9


WebVisualizer(window_uid='window_8')

# Final Result

![obstacle_fig](./gif/data_1.gif)